In [ ]:
# only for macbook with AMD GPU
import os
import plaidml.keras
plaidml.keras.install_backend()
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"


In [21]:
! gdown --id 1_-DZVz5AKqrxqpvh-s7Q3tKgirJhJzRn

Downloading...
From: https://drive.google.com/uc?id=1_-DZVz5AKqrxqpvh-s7Q3tKgirJhJzRn
To: /content/filter_new.zip
113MB [00:00, 184MB/s]


In [22]:
!pwd


/content


In [ ]:
!unzip  "/content/filter_new.zip"

Archive:  /content/filter_new.zip
replace filter/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/filter/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [1]:
import pandas as pd, matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
import cv2
from tqdm import tqdm_notebook as tqdm
import gc

In [3]:
train_dir = './filter/train_imgs_128'
valid_dir = './filter/valid_imgs_128'
test_dir = './filter/test_imgs_128'

In [4]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
import cv2
import os
from tqdm.notebook import tqdm
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten , ELU
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, MaxPool2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import keras as keras
import keras.backend as K
from keras.models import load_model

In [5]:
# # if no split
train_datagen = ImageDataGenerator(rescale=1./255.0)


valid_datagen = ImageDataGenerator(rescale=1./255.0)
test_datagen = ImageDataGenerator(rescale=1./255.0)


In [15]:
batch_size = 16

In [16]:
# if no split
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=batch_size,
    subset='training',class_mode='categorical', color_mode ='grayscale') # set as training data

validation_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='categorical',color_mode ='grayscale') # set as validation data
trest_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='categorical',color_mode ='grayscale') # set as validation data

Found 16320 images belonging to 5 classes.
Found 5440 images belonging to 5 classes.
Found 5440 images belonging to 5 classes.


In [17]:
model = None

In [18]:
model = Sequential()

model.add(Conv2D(64, (3,3),strides = (1,1), input_shape = (256,256,1),kernel_initializer='glorot_uniform'))

model.add(ELU())

model.add(BatchNormalization())

model.add(Conv2D(64, (3,3),strides = (1,1),kernel_initializer='glorot_uniform'))

model.add(ELU())

model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2, 2), strides= (2,2)))

model.add(Conv2D(128, (3,3),strides = (1,1),kernel_initializer='glorot_uniform'))

model.add(ELU())

model.add(BatchNormalization())

model.add(Conv2D(128, (3,3),strides = (1,1),kernel_initializer='glorot_uniform'))

model.add(ELU())

model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2, 2), strides= (2,2)))

model.add(Conv2D(256, (3,3),strides = (1,1),kernel_initializer='glorot_uniform'))

model.add(ELU())

model.add(BatchNormalization())

model.add(Conv2D(256, (3,3),strides = (1,1),kernel_initializer='glorot_uniform'))

model.add(ELU())

model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2, 2), strides= (2,2)))

model.add(Flatten())

model.add(Dense(2048))

model.add(ELU())

model.add(BatchNormalization())

model.add(Dropout(0.5))

model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

ResourceExhaustedError: ignored

In [ ]:
print(model.summary())

In [ ]:
import tensorflow as tf

In [ ]:
weight = {0: 0.6470028544243578,
 1: 0.8965062623599209,
 2: 1.3658046698468491,
 3: 0.803426377196869,
 4: 2.761421319796954}

In [ ]:
mcp_save =keras.callbacks.ModelCheckpoint('./best/mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
mcp_save_acc =keras.callbacks.ModelCheckpoint('./best/mdl_wts.hdf5', save_best_only=True, monitor='val_acc', mode='max')


In [ ]:
history = model.fit_generator(
    train_generator,
    class_weight = weight,
    callbacks = [mcp_save,mcp_save_acc],
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=25)

In [ ]:
print(keras.__version__)

In [ ]:
model.save("CNN_10_E_A_97.h5")

In [ ]:
!ls -as


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
model = load_model('/content/drive/MyDrive/CNN_10_E_A_97.h5')

In [ ]:
test_datagen

In [ ]:
scores = model.evaluate_generator(trest_generator) #1514 testing images


In [ ]:
scores